In [1]:
import csv
import xml.etree.ElementTree as et
import numpy as np
from datetime import datetime as dt
import pandas as pd


tree = et.ElementTree(file = "data/f24-sample-1.xml")
games = tree.getroot()

match_details = games[0].attrib
match_details

{'id': '1059926',
 'away_score': '2',
 'away_team_id': '31',
 'away_team_name': 'Crystal Palace',
 'competition_id': '8',
 'competition_name': 'English Premier League',
 'game_date': '2020-01-18T15:00:00',
 'home_score': '2',
 'home_team_id': '43',
 'home_team_name': 'Manchester City',
 'matchday': '23',
 'period_1_start': '2020-01-18T15:00:06',
 'period_2_start': '2020-01-18T16:02:11',
 'season_id': '2019',
 'season_name': 'Season 2019/2020'}

In [2]:
print("%s v %s, %s %s" % (match_details["home_team_name"],
                          match_details["away_team_name"],
                          match_details["competition_name"][8:],
                          match_details["season_name"][7:]))

print( "Date: %s" % dt.strftime(dt.strptime(match_details["game_date"], '%Y-%m-%dT%H:%M:%S'),
                               "%A %d %B %Y"))
print( "Kick-off: %s" % dt.strftime(dt.strptime(match_details["game_date"], '%Y-%m-%dT%H:%M:%S'),
                               "%I%p").lstrip("0"))

Manchester City v Crystal Palace, Premier League 2019/2020
Date: Saturday 18 January 2020
Kick-off: 3PM


In [3]:
team_dict = {match_details["home_team_id"]: match_details["home_team_name"],
             match_details["away_team_id"]: match_details["away_team_name"]}

print(team_dict)

{'43': 'Manchester City', '31': 'Crystal Palace'}


In [4]:
passes_x = []
passes_y = []
passes_outcome = []
passes_min = []
passes_sec = []
passes_period = []
passes_team = []
passes_x_end = []
passes_y_end = []
passes_length = []
passes_angle = []
passes_zone = []
pass_real = []
pass_player = []


for game in games:
    for event in game:
        if event.attrib.get("type_id") == '1':
            passes_x.append(event.attrib.get("x"))
            passes_y.append(event.attrib.get("y"))
            passes_outcome.append(event.attrib.get("outcome"))
            passes_min.append(event.attrib.get("min"))
            passes_sec.append(event.attrib.get("sec"))
            passes_period.append(event.attrib.get("period_id"))
            passes_team.append(team_dict[event.attrib.get("team_id")])
            pass_player.append(event.attrib.get("player_id"))

            for q in event:
                qualifier = q.attrib.get("qualifier_id")
                
                if qualifier == "140":
                    passes_x_end.append(q.attrib.get("value"))
                if qualifier == "141":
                    passes_y_end.append(q.attrib.get("value"))
                if qualifier == "212":
                    passes_length.append(q.attrib.get("value"))
                if qualifier == "213":
                    passes_angle.append(q.attrib.get("value"))
                if qualifier == "56":
                    passes_zone.append(q.attrib.get("value"))

data = np.array(list(zip(passes_team, pass_player, passes_period, passes_min, passes_sec, passes_zone, passes_x, 
                         passes_y, passes_x_end, passes_y_end, passes_length, passes_angle, 
                         passes_outcome)))

fieldnames = ["team", "player", "period", "min", "sec", "pass zone", "x", "y", "x_end", "y_end",
              "pass length", "pass angle", "outcome"]

df = pd.DataFrame(data, columns=fieldnames)

df

,team,player,period,min,sec,pass zone,x,y,x_end,y_end,pass length,pass angle,outcome
0,Manchester City,20664,1,0,0,Back,50.0,50.0,38.1,56.0,13.1,2.8,1
1,Manchester City,59859,1,0,2,Right,38.3,54.6,67.9,7.7,44.5,5.5,0
2,Crystal Palace,173954,1,0,5,Back,32.6,92.0,41.8,91.7,9.7,6.3,0
3,Manchester City,61366,1,0,6,Right,61.3,8.0,74.8,16.8,15.4,0.4,0
4,Crystal Palace,19419,1,0,9,Back,27.4,82.3,44.3,92.9,19.2,0.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,Manchester City,59859,2,95,15,Center,61.6,59.4,58.7,33.0,18.2,4.5,1
1033,Manchester City,121145,2,95,18,Right,64.6,36.3,78.5,17.9,19.2,5.6,0
1034,Crystal Palace,55494,2,95,23,Back,20.2,70.9,41.7,92.1,26.8,0.6,1
1035,Crystal Palace,19419,2,96,0,Left,51.4,89.4,81.7,100.0,32.8,0.2,0
